In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from math import sqrt
from numpy import split
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import RepeatVector
from keras.layers import GRU
from keras.layers import TimeDistributed
from keras_tuner import RandomSearch
from keras_tuner import Hyperband
from keras_tuner import BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression

In [2]:
import random
random.seed(123)

In [3]:
%run Funciones_Cons.ipynb

In [5]:
def pruebas(interval, block, variable_mode, n_obs_antes, n_pred):
    res_time = interval
    edificio = block
    variables = variable_mode
    data = pd.read_csv('data-model-consumo' + edificio + '-' + res_time + '.csv', sep=";", index_col=0)
    
    if variables == 1:
        data = data[['dif_cons', 'V4']]
    else: 
        data = data.drop(columns='Fecha')
    df = data
    
    per_train = 0.7
    per_test = 0.15  
    train, test, val, scaler_x, scaler_y = split_dataset(df.values, per_train, per_test)
    
    n_input = n_obs_antes
    n_out = 1
    steps_pred = n_pred
    train_x, train_y = to_supervised(train, n_input, n_out, steps_pred)
    test_x, test_y = to_supervised(test, n_input, n_out, steps_pred)
    val_x, val_y = to_supervised(val, n_input, n_out, steps_pred)
    
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

    def build_model(hp):
        model = Sequential()
        hp_units = hp.Choice('input_unit',[32,64,128])
        model.add(GRU(units = hp_units, input_shape=(n_timesteps, n_features)))
        model.add(RepeatVector(n_outputs))   
        for i in range(hp.Int('n_layers', 1, 2)):
            units_i = hp.Choice(f'dense_{i}_units',[32,64,128])
            model.add(GRU(units = units_i, return_sequences=True))
        hp_dropout = hp.Float('Dropout_rate',min_value=0,max_value=0.4,step=0.1)
        model.add(Dropout(hp_dropout))
        model.add(TimeDistributed(Dense(units = hp_units)))
        model.add(TimeDistributed(Dense(n_outputs)))
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        model.compile(loss='mae', optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate))
        return model
    
    tuner= BayesianOptimization(build_model,
                                 objective='val_loss', max_trials=15, num_initial_points=2, seed = 123, overwrite=True)

    stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10)
    tuner.search(x=train_x, y=train_y, epochs=200, batch_size=128, validation_data=(val_x,val_y), verbose=2,
            shuffle=False, callbacks=[stop_early])
    model = tuner.get_best_models(num_models=1)[0]
    
    prediction = model.predict(test_x)
    prediction = prediction.reshape(prediction.shape[0],prediction.shape[1])
    prediction = scaler_y.inverse_transform(prediction)
    test_y = scaler_y.inverse_transform(test_y)
    
    cvrmse, r2, rmse = metrics(test_y, prediction)
    
    return model, cvrmse, r2, rmse, test_y, prediction

In [6]:
obs_antes = [1,2,4]  #1,2,4
pred_steps = 1    #1
block = ['A','B','C']
interval = ['60T','120T']
var_mode = [1,2] #1 = dif_cons + V4, #2 = all

for i in block:
    for j in interval:
        for k in obs_antes:
            for l in var_mode:
                model, cvrmse, r2, rmse, test_y, prediction = pruebas(j, i, l, k, pred_steps)
                model.save('model_cons_gru_' + i + '_0'+ str(l) + '_pred' + j + '_prev' + str(k) + '.h5')
                results_metrics = pd.read_csv('model-cons-results_metrics.csv', sep=";")
                add_metrics = {'Modelo': 'GRU', 'Edificio': i,'Variables': '0'+str(l), 'pred': j, 'n_obs_antes': k, 'CVRMSE': cvrmse, 'R2': r2, 'RMSE': rmse}
                results_metrics = results_metrics.append(add_metrics, ignore_index = True)
                results_metrics.to_csv('model-cons-results_metrics.csv', sep=";", index=False)
                results_preds = pd.read_csv('model-cons-results_preds.csv', sep=";")
                prediction = prediction.round(2)
                add_preds = pd.DataFrame({'Prueba': np.repeat('GRU-' + i + '-' + str(l) + '-' + j +'-' + str(k), len(test_y)),
                                        'test_y': test_y.tolist(),'prediction': prediction.tolist()})
                results_preds = pd.concat([results_preds, add_preds])
                results_preds.to_csv('model-cons-results_preds.csv', sep=";", index=False)

Trial 15 Complete [00h 00m 46s]
val_loss: 0.2148205190896988

Best val_loss So Far: 0.17817559838294983
Total elapsed time: 00h 11m 59s
INFO:tensorflow:Oracle triggered exit
